In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [39]:
##  LOad the dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head(6)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1


In [40]:
## first 3 featurere will not play anay role
## preprocess the data 
## Drop irrelevant col
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [41]:
data.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
## Geography and Gender are Categorical Varible
## ENCODE categorical varabile
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [43]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [44]:
## if we use label encoder here in country it iwll think one country is greater than other
# so for this we will use OHE 

## OHE --"Geography"
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [45]:
### convert into np array
onehot_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [46]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [47]:
## gettiing them in form of df
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out())

In [48]:
## combine all cols of OHE with original 

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [49]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [50]:
## how we have sane oHE and label encoding we nedd to ssve in pickle file as
### we need to save them in main model for pred

In [51]:
## SAve the encoder and sclaer
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [52]:
## divide daaset into dep and indep fetre
data.head()

## EXITED is col for dep fwature

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [53]:
X=data.drop('Exited',axis=1)
y=data['Exited']

In [54]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scaling feature
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [55]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [56]:
data


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


# ***ANN Implementation***

In [57]:
X_train.shape[1],

(12,)

In [58]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [59]:
## build ou ann model
models=Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
        ## input_shape will be for only first hidden layer coonected with i/p layer
        Dense(32,activation='relu'), ## HL2
        Dense(1,activation='sigmoid') ## ouput layer

    ]
)

In [60]:
models.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()
opt

In [62]:
## compile the modle
models.compile(optimizer=opt,loss=loss,metrics=['accuracy'])
## for multiclass sparse cross entropy

In [63]:
## setup tensor board - to visaulize all the logs while training the model
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [64]:
## Setup Earlystopping - we can  train it for many epochs
## if after 20 epochs our model training good so we need not to train for 100 epochs 
## means our loss not decreasing much so stop traing early
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [65]:
### Training the model 

In [66]:
history=models.fit(X_train,y_train,
                   validation_data=(X_test,y_test),
                   epochs=100,
                   callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3992 - accuracy: 0.8349 - val_loss: 0.3660 - val_accuracy: 0.8445
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3560 - accuracy: 0.8539 - val_loss: 0.3711 - val_accuracy: 0.8565
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3477 - accuracy: 0.8574 - val_loss: 0.3442 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3478 - accuracy: 0.8606 - val_loss: 0.3609 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3431 - accuracy: 0.8585 - val_loss: 0.3516 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3408 - accuracy: 0.8608 - val_loss: 0.3648 - val_accuracy: 0.8535
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3396 - accuracy: 0.8604 - val_loss: 0.3426 - val_accuracy: 0.8610

In [67]:
models.save('model.h5')

/home/abc/Desktop/ANN/annenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [68]:
# .h5 file is compatible with keras so we using .h5 file

In [69]:
## load tensor board exetnsion
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
!tensorboard --logdir logs/fit --port 6006

/home/abc/Desktop/ANN/annenv/lib/python3.11/site-packages/tensorboard/default.py:30: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-11-11 18:12:50.239136: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-11 18:12:50.241664: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-11 18:12:50.291784: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2

In [71]:
%tensorboard --logdir logs/fit
## we need to give folder name nere logs/foldername

Reusing TensorBoard on port 6006 (pid 6788), started 0:10:02 ago. (Use '!kill 6788' to kill it.)

In [72]:
## we were able to check epoch accuracy /loss... mamy many things using tensor board 

In [73]:
## Load the pickle file for prediction on new data
## in new file
